# Introducción
¡Bienvenido al curso de aprendizaje automático intermedio!

Si tienes experiencia en aprendizaje automático y te gustaría aprender cómo mejorar rápidamente la calidad de tus modelos, ¡estás en el lugar correcto! En este curso, acelerará su experiencia en aprendizaje automático aprendiendo a:

* Abordar los tipos de datos que a menudo se encuentran en conjuntos de datos del mundo real (valores faltantes, variables categóricas).
* Diseñar canales para mejorar la calidad de su código de aprendizaje automático.
* Utilizar técnicas avanzadas para la validación de modelos (validación cruzada), construir modelos de última generación que se utilizan ampliamente para ganar competencias de Kaggle (XGBoost), y
* Evite errores comunes e importantes en la ciencia de datos (fugas).

 

## Requisitos previos
Está listo para este curso si ya ha creado un modelo de aprendizaje automático y está familiarizado con temas como la validación de modelos, el ajuste insuficiente y excesivo, y los bosques aleatorios. 

# 1. Valores faltantes

En este tutorial, aprenderá tres enfoques para lidiar con valores faltantes. Luego comparará la efectividad de estos enfoques en un conjunto de datos del mundo real. 
 
Hay muchas formas en que los datos pueden terminar con valores faltantes. Por ejemplo,

* Una casa de 2 dormitorios no incluirá el valor del tamaño de un tercer dormitorio.
* Un encuestado puede optar por no compartir sus ingresos.

La mayoría de las bibliotecas de aprendizaje automático (incluida scikit-learn) dan un error si intentas construir un modelo usando datos con valores faltantes. Por lo tanto, deberá elegir una de las estrategias siguientes.

### 1. Una opción simple: eliminar columnas con valores faltantes

La opción más sencilla es eliminar las columnas con valores faltantes. A menos que falten la mayoría de los valores en las columnas eliminadas, el modelo pierde acceso a mucha información (¡potencialmente útil!) con este enfoque.

Como ejemplo extremo, considere un conjunto de datos con 10.000 filas, donde a una columna importante le falta una sola entrada. ¡Este enfoque eliminaría la columna por completo!

### 2. Una mejor opción: la imputación
La imputación completa los valores faltantes con algún número. Por ejemplo, podemos completar el valor medio en cada columna.

El valor imputado no será exactamente correcto en la mayoría de los casos, pero generalmente conduce a modelos más precisos que los que se obtendrían si eliminara la columna por completo.

### 3. Una extensión de la imputación

La imputación es el enfoque estándar y normalmente funciona bien. Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (que no se recopilaron en el conjunto de datos). O las filas con valores faltantes pueden ser únicas de alguna otra manera. En ese caso, su modelo haría mejores predicciones al considerar qué valores faltaban originalmente.

En este enfoque, imputamos los valores faltantes, como antes. Y, además, para cada columna a la que le faltan entradas en el conjunto de datos original, agregamos una nueva columna que muestra la ubicación de las entradas imputadas.

En algunos casos, esto mejorará significativamente los resultados. En otros casos, no ayuda en absoluto.

## Ejemplo
Trabajaremos con el conjunto de datos de Melbourne Housing. Nuestro modelo utilizará información como la cantidad de habitaciones y el tamaño del terreno para predecir el precio de la vivienda.

No nos centraremos en el paso de carga de datos. En cambio, puedes imaginar que estás en un punto en el que ya tienes los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('/kaggle/input/melbourne-housing-snapshot/melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

### Definir función para medir la calidad de cada enfoque

Definimos una función score_dataset() para comparar diferentes enfoques para tratar con valores faltantes. Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio.

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Puntuación del método 1 (eliminar columnas con valores faltantes)
Dado que estamos trabajando con conjuntos de entrenamiento y validación, tenemos cuidado de eliminar las mismas columnas en ambos DataFrames.

In [12]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
 

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Puntuación del Método 2 (Imputación)
A continuación, usamos SimpleImputer para reemplazar los valores faltantes con el valor medio en cada columna.

Aunque es simple, completar el valor medio generalmente funciona bastante bien (pero esto varía según el conjunto de datos). Si bien los estadísticos han experimentado con formas más complejas de determinar los valores imputados (como la imputación de regresión, por ejemplo), las estrategias complejas generalmente no brindan ningún beneficio adicional una vez que se conectan los resultados a modelos sofisticados de aprendizaje automático.

In [13]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
 

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns



print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Vemos que el Método 2 tiene un MAE más bajo que el Método 1, por lo que el Método 2 funcionó mejor en este conjunto de datos.

### Puntuación del método 3 (una extensión de la imputación)
A continuación, imputamos los valores faltantes y al mismo tiempo realizamos un seguimiento de qué valores se imputaron.

In [18]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
 

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))
 

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Como podemos ver, el Método 3 tuvo un desempeño ligeramente peor que el Método 2.

### Entonces, ¿por qué la imputación funcionó mejor que eliminar las columnas?
Los datos de entrenamiento tienen 10864 filas y 12 columnas, donde tres columnas contienen datos faltantes. Para cada columna, faltan menos de la mitad de las entradas. Por lo tanto, eliminar las columnas elimina mucha información útil, por lo que tiene sentido que la imputación funcione mejor.

In [19]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Conclusión
Como es común, imputar valores faltantes (en los Métodos 2 y 3) arrojó mejores resultados, en comparación con cuando simplemente eliminamos las columnas con valores faltantes (en el Método 1).